# Foodle Maps

This notebook gives practical examples of the potentiality of Foodle Maps.
First, load the necessary packages to perform SPARQL queries:

In [20]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://127.0.0.1:3030/FoodleMaps")

Then, we can start with a simple query: show all the restaurants in the KB.

In [21]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    
    SELECT ?restaurant 
    WHERE { ?restaurant a schema:FoodEstablishment  }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["restaurant"]["value"])

http://www.semanticweb.org/FoodleMaps#Trattoria_dell_orsa


... or, we want to see some of the available dishes.

In [22]:
sparql.setQuery("""
    PREFIX schema: <http://schema.org/>
    
    SELECT ?dish 
    WHERE { ?dish a schema:MenuItem  }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["dish"]["value"])

http://www.semanticweb.org/FoodleMaps#Pasta_al_pomodoro
http://www.semanticweb.org/FoodleMaps#Pasta_alla_carbonara
http://www.semanticweb.org/FoodleMaps#Merluzzo_alla_livornese
http://www.semanticweb.org/FoodleMaps#Filetto_al_pepe_verde
http://www.semanticweb.org/FoodleMaps#Tiramisu
